In [1]:
import numpy as np
import igl
import meshplot as mp

In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [3]:
def sample_code1():
    pi, v = igl.read_triangle_mesh("data/cat.off")
    pi /= 10
    ni = igl.per_vertex_normals(pi, v)
    mp.plot(pi, shading={"point_size": 8})

sample_code1()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [4]:
# returns index of a point X which is closest to point in points
def find_closest_point(point, points) -> np.array:
    return np.argmin(
        np.linalg.norm(points - point, axis=1)
    )


In [5]:
# Part 1 - Constraints
def get_constraints(v, n, eps):
    points: list = []  # constraint pointss
    values: list = []  # function values

    for i in range(v.shape[0]): 
        point: np.array = v[i]
        normal: np.array = n[i] / np.linalg.norm(n[i])  # normalized normal

        # On surface points and values
        points.append(point)
        values.append(0)

        # Off surface constraints and values
        for eps_sign in [1, -1]:
            
            eps_value: int = eps * eps_sign
            candidate_p: np.array = point + eps_value * normal

            # cehcks that p_i is closest to candidate point
            while find_closest_point(candidate_p, v) != i:
                eps_value /= 2 # halve the eps and keep searchingg
                candidate_p = point + eps_value * normal

            points.append(candidate_p)
            values.append(eps_value)

    return np.array(points),  np.array(values)


In [6]:
# CAT values
v, faces = igl.read_triangle_mesh("data/cat.off")
v /= 10  
n = igl.per_vertex_normals(v, faces)
# EPS
box_max = np.max(v, axis=0)
box_min = np.min(v, axis=0)
box_diag = np.linalg.norm(box_max-box_min)
eps = 0.01 * box_diag

# Getting constraints
constraint_points, constraint_values = get_constraints(v, n, eps)

# Separate points by constraint 
on_surface_points = constraint_points[constraint_values == 0]
outside_points = constraint_points[constraint_values > 0]
inside_points = constraint_points[constraint_values < 0]

# Visualize
plot = mp.plot(on_surface_points, shading={"point_color": "blue", "point_size": 8})  
plot.add_points(outside_points, shading={"point_color": "red", "point_size": 8})   
plot.add_points(inside_points, shading={"point_color": "green", "point_size": 8})   


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

2

# MLS function

In [7]:
# Parameters
def sample_code2():
    bbox_min = np.array([-1., -1., -1.])
    bbox_max = np.array([1., 1., 1.])
    bbox_diag = np.linalg.norm(bbox_max - bbox_min)

    n = 10

    #------

    # Generate grid n x n x n

    x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

    #Compute implicit sphere function
    center = np.array([0., 0., 0.])
    radius = 1
    fx = np.linalg.norm(x-center, axis=1) - radius

    #------

    # Treshold fx to visualize inside outside

    ind = np.zeros_like(fx)
    ind[fx >= 0] = 1
    ind[fx < 0] = -1
    mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800})

# sample_code2()

In [8]:
# returns indecies of a point X which are within distance h in points
def closest_points(point, points, h) -> np.array:
    distances: np.array = np.linalg.norm(points - point, axis=1) 
    return np.argwhere(distances < h).flatten()

def wendland_weight(r, h):
    if r >= h:
        return 0
    return (1 - r / h)**4 * (4 * r / h + 1)

def polynomial_basis(P, degree):
    num = P.shape[0]  
    x, y, z = P[:, 0], P[:, 1], P[:, 2]  

    # degree 0 - just 1s
    if degree == 0:
        return np.ones((num, 1)) 

    # degree 1 - [1, x, y, z]
    if degree == 1:
        return np.column_stack(
            (np.ones(num), x, y, z)
        )  

    # degree 2 -
    if degree == 2:
        return np.column_stack((
            np.ones(num),  
            x, y, z,  
            x * y, y * z, z * x,  
            x ** 2, y ** 2, z ** 2 
        ))
    
    raise RuntimeError

In [9]:
def mls_interpolation(points, constraint_points, constraint_values, wendland_radius, poly_degree):
    poly_coef_double_map = [1, 4, 10]  # Required number of points
    fx = np.zeros(points.shape[0]) # new values

    for i, pi in enumerate(points):
        closest_idx = closest_points(pi, constraint_points, wendland_radius)
        # number of points must be more than double of poly degree
        if len(closest_idx) < poly_coef_double_map[poly_degree]:
            fx[i] = 999999  # Assign large value for outside
            continue

        P: np.array = constraint_points[closest_idx] - pi

        # Ccompute weights using wendland function
        distances: np.array = np.linalg.norm(P, axis=1)
        weights: np.array = np.array([wendland_weight(r, wendland_radius) for r in distances])

        # polynomial basis 
        Basis = polynomial_basis(P, poly_degree)
        # Weight matrix
        Weight = np.diag(weights)

        # We are trying to solve A * c = b where A and b are known
        A = Basis.T @ Weight @ Basis
        b: np.array = Basis.T @ Weight @ constraint_values[closest_idx]


        try:
            c = np.linalg.solve(A, b)  # solve A * c = b
            fx[i] = (polynomial_basis(np.array([[0, 0, 0]]), poly_degree) @ c).item() # solve f(x) = B * c

            # fx[i] = c[0]
            #fx[i] = b.dot(c)

        except: #LinAlgError: Singular matrix with polydefree != 0
            fx[i] = 999999


    return fx

In [10]:
# cat points
constraint_points, constraint_values = get_constraints(v, n, eps)

# config
wendland_radius =  15
resolution = 50

# construct grid
bbox_max = np.max(constraint_points, axis=0)
bbox_min = np.min(constraint_points, axis=0)
bbox_diag = np.linalg.norm(bbox_max-bbox_min) 
grid, T = tet_grid((resolution, resolution, resolution), bbox_min - 0.03 * bbox_diag, bbox_max + 0.03 * bbox_diag)


#
def plot_mls(grid, constraint_points, constraint_values, wendland_radius, poly_degree=0):
    fx = mls_interpolation(grid, constraint_points, constraint_values, wendland_radius, poly_degree)

    # Separate points by values 
    outside_points = grid[fx >= 0]
    inside_points = grid[fx < 0]

    # Visuaze
    plot = mp.plot(outside_points, shading={"point_color": "green", "point_size": 2})  
    plot.add_points(inside_points, shading={"point_color": "red", "point_size": 5})   

    return fx

# Marching to extract surface

In [11]:
def filter_largest_component(sf):
    components = igl.facet_components(sf)

    largest_comp = np.bincount(components).argmax()

    return np.array( 
        [face for face_i, face in enumerate(sf) if components[face_i] == largest_comp]
    )



## MLS Interpolation with Poly Degree = 0

In [12]:
print("Poly degree is 0")
fx = plot_mls(grid, constraint_points, constraint_values, wendland_radius, poly_degree=0)

# Marcing tet to extract surface
sv, sf, _, _ = igl.marching_tets(grid, T, fx, 0)
filtered_sf = filter_largest_component(sf)
mp.plot(sv, filtered_sf, shading={"wireframe": True})

Poly degree is 0


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(4.9494981…

## MLS Interpolation with Poly Degree = 1

In [13]:
print("Poly degree is 1")
fx = plot_mls(grid, constraint_points, constraint_values, wendland_radius, poly_degree=1)

# Marcing tet to extract surface
sv, sf, _, _ = igl.marching_tets(grid, T, fx, 0)
filtered_sf = filter_largest_component(sf)
mp.plot(sv, filtered_sf, shading={"wireframe": True})

Poly degree is 1


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.6967134…

## MLS Interpolation with Poly Degree = 2

In [14]:
print("Poly degree is 2")
fx = plot_mls(grid, constraint_points, constraint_values, wendland_radius, poly_degree=2)

# Marcing tet to extract surface
sv, sf, _, _ = igl.marching_tets(grid, T, fx, 0)
filtered_sf = filter_largest_component(sf)
mp.plot(sv, filtered_sf, shading={"wireframe": True})

Poly degree is 2


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

# Spatial Index

#  Using a non-axis-aligned grid


In [15]:
def align(v_orig):
    #https://www.turing.com/kb/guide-to-principal-component-analysis
    center = np.mean(v_orig, axis=0)
    v = v_orig - center 
    
    cov_matrix = np.cov(v.T)

    _, eigenvectors = np.linalg.eigh(cov_matrix)

    v = v @ eigenvectors
    n = igl.per_vertex_normals(v, faces)
    return v, n

def luigi(v):
    resolution = 20

    bbox_max = np.max(v, axis=0)
    bbox_min = np.min(v, axis=0)
    bbox_diag = np.linalg.norm(box_max-box_min) 
    grid, T = tet_grid((resolution, resolution, resolution), bbox_min - 0.03 * bbox_diag, bbox_max + 0.03 * bbox_diag)

    p = mp.plot(grid, shading={"point_size": 1, "point_color": "green", "width": 800, "height": 800})
    p.add_points(v, shading={"point_size": 3, "point_color": "red"})

    # ----------------
    v2, n2 = align(v)

    bbox_max = np.max(v2, axis=0)
    bbox_min = np.min(v2, axis=0)
    bbox_diag = np.linalg.norm(box_max-box_min) 
    grid, T = tet_grid((resolution, resolution, resolution), bbox_min - 0.03 * bbox_diag, bbox_max + 0.03 * bbox_diag)
    
    p = mp.plot(grid, shading={"point_size": 1, "point_color": "green", "width": 600, "height": 600})
    p.add_points(v2, shading={"point_size": 3, "point_color": "red"})

    return v2, n2



In [16]:
v1, faces = igl.read_triangle_mesh("data/luigi.off")
v1 /= 10
n1 = igl.per_vertex_normals(v1, faces)

v2, n2 = luigi(v1)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.1287999…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.019215…

In [17]:
# Unfortuntaly this part takes forever to run


# constraint_points, constraint_values = get_constraints(v2, -n2, eps)


# # config
# resolution = 40
# bbox_max = np.max(v1, axis=0)
# bbox_min = np.min(v1, axis=0)
# bbox_diag = np.linalg.norm(box_max-box_min) 

# grid, T = tet_grid((resolution, resolution, resolution), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

# wendland_radius =  12

# fx = plot_mls(grid, constraint_points, constraint_values, wendland_radius, poly_degree=0)

# # Marcing tet to extract surface
# sv, sf, _, _ = igl.marching_tets(grid, T, fx, 0)
# mp.plot(sv, sf, shading={"wireframe": True})